## oct 8 new plan

Use this!! https://github.com/damian0815/finetune-clip-huggingface/blob/main/huggingface_finetune_clip_runner.ipynb

In [1]:
!pip install pandas requests

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/ff/5a/c7359edec58500b35da8dc40a69ea7b0a3be48a479e1c91e8e8d0a2d9aa7/pandas-2.1.1-cp310-cp310-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for numpy>=1.22.4 from https://files.pythonhosted.org/packages/5c/ff/0e1f31c70495df6a1afbe98fa237f36e6fb7c5443fcb9a53f43170e5814c/numpy-1.26.0-cp310-cp310-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 347.8 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 1.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import requests
import pandas as pd

url = 'https://raw.githubusercontent.com/opencontext/archaeology-images-ai/main/json_data/artifact_images_w_descriptions.json'
data = requests.get(url).json()
df = pd.json_normalize(data)  # convert json to pandas DataFrame

In [3]:
df.rename(columns={'image_file__uri': 'image'}, inplace=True)
df

,image,media__uri,image_genre,image_type,subject__item_class__label,context___1,context___2,context___3,time_range,has_type,consists_of,origin_place,has_taxonomic_identifier,has_anatomical_identification,temporal_coverage,project_specific_descriptions
0,https://iiif.archivelab.org/iiif/opencontext-1...,https://opencontext.org/media/a9cedbad-e25b-4f...,archaeology,artifact,Object,Asia,Turkey,Domuztepe,6500 BCE to 5500 BCE,seals (artifacts),rock (inorganic material),None,None,None,None,"Artifact Name: Stamp Seal \n Material: Stone, ..."
1,https://iiif.archivelab.org/iiif/opencontext-1...,https://opencontext.org/media/1bbbca07-82f3-46...,archaeology,artifact,Object,Asia,Turkey,Domuztepe,6500 BCE to 5500 BCE,seals (artifacts),soapstone,None,None,None,None,Artifact Name: Stamp Seal \n Material: Steatit...
2,https://iiif.archivelab.org/iiif/opencontext-1...,https://opencontext.org/media/2062e3fa-41e2-d7...,archaeology,artifact,Object,Asia,Turkey,Domuztepe,6500 BCE to 5500 BCE,seals (artifacts),rock (inorganic material),None,None,None,None,"Artifact Name: Stamp Seal \n Material: Stone, ..."
3,https://iiif.archivelab.org/iiif/opencontext-1...,https://opencontext.org/media/2dc18114-4ddf-7c...,archaeology,artifact,Object,Asia,Turkey,Domuztepe,6500 BCE to 5500 BCE,pendants (jewelry),chert,None,None,None,None,Artifact Name: Pendant \n Material: Chert/Flint
4,https://iiif.archivelab.org/iiif/opencontext-1...,https://opencontext.org/media/d7e8b4e5-be3b-44...,archaeology,artifact,Object,Asia,Turkey,Domuztepe,6500 BCE to 5500 BCE,nails (fasteners),iron (metal),None,None,None,None,Artifact Name: Nail \n Material: Iron \n Dispo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72357,https://artiraq.org/static/opencontext/pettegr...,https://opencontext.org/media/d3620d27-cb41-44...,archaeology,artifact,Object,Europe,Greece,Corinthia,,None,None,None,None,None,None,"Chronotype: Fineware, Late Helladic I-IIA \n Z..."
72358,https://artiraq.org/static/opencontext/pettegr...,https://opencontext.org/media/41f03708-baa5-4d...,archaeology,artifact,Object,Europe,Greece,Corinthia,,None,None,None,None,None,None,"Chronotype: Fineware, Late Helladic I-IIA \n Z..."
72359,https://artiraq.org/static/opencontext/pettegr...,https://opencontext.org/media/453e04b2-7905-4e...,archaeology,artifact,Object,Europe,Greece,Corinthia,,None,None,None,None,None,None,"Chronotype: Fineware, Late Helladic I-IIA \n Z..."
72360,https://artiraq.org/static/opencontext/interna...,https://opencontext.org/media/1840719d-2934-48...,archaeology,artifact,Object,Off World,International Space Station,Zvezda Service Module,,icons (devotional images); religions and reli...,None,None,None,None,None,Location: Top center \n Item type: Icon \n Sec...


In [4]:
import os
import random
import requests

# Shuffle your DataFrame
df = df.sample(frac=1, random_state=67)

# Split the data into training and testing datasets
train_images = df.iloc[:500]
test_images = df.iloc[500:600]

def download_images(df, dir_name):
    os.makedirs(dir_name, exist_ok=True)
    url_errors=[]
    for _, row in df.iterrows():
        url = row['image']
        # Get the extension for the image file from the URL
        extension = url.split('.')[-1]
        # Get the UUID for the media file
        media_uuid = row['media__uri'].split('/')[-1]
        file_name = f'{media_uuid}.{extension}'
        file_path = os.path.join(dir_name, file_name)

        try:
            response = requests.get(url, timeout=5)
            response.raise_for_status()
        except (requests.exceptions.RequestException, requests.exceptions.Timeout):
            print(f'An error occurred while fetching: {url}')
            url_errors.append(url)
            continue

        with open(file_path, 'wb') as img_file:
            img_file.write(response.content)
    
    print(f"Downloaded images in directory '{dir_name}'. Number of url errors: {len(url_errors)}")

download_images(train_images, 'images') # downloads training images to 'images' folder
download_images(test_images, 'testing') # downloads testing images to 'testing' folder

An error occurred while fetching: https://artiraq.org/static/opencontext/poggio-civitate/preview/photos/20030129PROFILE.jpg
An error occurred while fetching: https://iiif.archivelab.org/iiif/opencontext-24-20040043ajpg/full/675,/0/default.jpg
An error occurred while fetching: https://iiif.archivelab.org/iiif/opencontext-22-f-2-2042-1-6-p-4jpg/full/675,/0/default.jpg
An error occurred while fetching: https://iiif.archivelab.org/iiif/opencontext-24-19960107-2009-003jpg/full/675,/0/default.jpg
An error occurred while fetching: https://iiif.archivelab.org/iiif/opencontext-14-b2009177-3jpg/full/675,/0/default.jpg
An error occurred while fetching: https://iiif.archivelab.org/iiif/opencontext-22-e-2-179-17-1-p-2jpg/full/675,/0/default.jpg
Downloaded images in directory 'images'. Number of url errors: 6
Downloaded images in directory 'testing'. Number of url errors: 0


In [ ]:
import re
import string
def create_metadata(df, dir_name):
    # Make a copy of the dataframe to avoid modifying the original one
    df = df.copy()

    # fill NaN values with empty string
    df.fillna('', inplace=True)

    # append values of required columns into new 'caption' column
    df['caption'] = 'A photograph of ' + df['consists_of'].astype(str) \
                    + ', ' + df['project_specific_descriptions'].astype(str) \
                    + ' dating to ' + df['time_range'].astype(str) \
                    + ' from ' + df['context___1'].astype(str) \
                    + ', ' + df['context___2'].astype(str) \
                    + ', ' + df['context___3'].astype(str)
    df['caption'] = df['caption'].replace('\n', ' ')
    df['caption'] = df['caption'].replace('False', ' ')
    df['caption'] = df['caption'].replace('True', ' ')
    # Remove all other punctuation
    df['caption'] = df['caption'].apply(lambda x: re.sub(r'[{}]'.format(string.punctuation), ' ', x)).str.strip()

    # Rewrite 'image' column to just contain the filename
    df['image'] = df.apply(lambda row: f"{row['media__uri'].split('/')[-1]}.{row['image'].split('.').pop()}", axis=1)

    # reshaping data to contain only 'image' and 'caption'
    df = df[['image', 'caption']]
    df.loc[:, 'image'] = dir_name + '/' + df['image'].astype(str)

    return df

df = create_metadata(train_images, 'images')
with open('train.json', 'w') as file:
    df.to_json(file, orient='records', lines=True)

testdf = create_metadata(test_images, 'testing')
with open('test.json', 'w') as file:
    testdf.to_json(file, orient='records', lines=True)

In [2]:
!pip install torchvision datasets Pillow
!pip install -q git+https://github.com/huggingface/transformers
!pip install accelerate -U

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/de/5c/63da40bcac00e06eae06b52b4a82ba089b44d2cbd232020fe9194ea75ea9/torchvision-0.16.0-cp310-cp310-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/09/7e/fd4d6441a541dba61d0acb3c1fd5df53214c2e9033854e837a99dd9e0793/datasets-2.14.5-py3-none-any.whl.metadata
  Obtaining dependency information for Pillow from https://files.pythonhosted.org/packages/55/93/9a8b06ca5917f246d1cd31351d5ec38651f8f5f9619c8db5751583f49ba3/Pillow-10.0.1-cp310-cp310-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for torch==2.1.0 from https://files.pythonhosted.org/packages/ab/6a/0debe1ec3c63b1fd7487ec7dd8fb1adf19898bef5a8dc151265d79ffd915/torch-2.1.0-cp310-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/5e/5d/97afbafd9d584ff1b45fcb354a479a3609bd97f91

In [3]:
# test loading it back in
from datasets import load_dataset
dataset = load_dataset("json", data_files="train.json")
print(f"first image: {dataset['train'][0]['image']}, caption: '{dataset['train'][0]['caption']}'")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

first image: images/a8ea6c5f-bdc5-45ca-9d41-9023a9152c1e.jpg, caption: 'A photograph of pottery  Fragment Noted  false 
 Fabric Category  Impasto 
 Object Type  Textile Related  Spindle Whorl 
 Decorative Technique  Stamped dating to 700 BCE to 535 BCE from Europe  Italy  Poggio Civitate'


In [5]:
!git clone https://github.com/damian0815/finetune-clip-huggingface.git

Cloning into 'finetune-clip-huggingface'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 19 (delta 6), reused 17 (delta 4), pack-reused 0
Receiving objects: 100% (19/19), 13.79 KiB | 336.00 KiB/s, done.
Resolving deltas: 100% (6/6), done.


In [6]:
!mkdir results

In [8]:
repo_id =  "openai/clip-vit-base-patch32"
output_folder = "results"
batch_size = 20
num_train_epochs = 60
out_json = "train.json"

In [ ]:
print(f"Finetuning {repo_id} for {num_train_epochs} epochs with batch size {batch_size}, and then saving output to {output_folder}.")
!python -W ignore finetune-clip-huggingface/huggingface_finetune_clip.py \
    --output_dir {output_folder} \
    --model_name_or_path {repo_id} \
    --train_file {out_json} \
    --image_column image \
    --overwrite_output_dir=True \
    --max_seq_length=77 \
    --num_train_epochs={num_train_epochs} \
    --caption_column caption \
    --remove_unused_columns=False \
    --do_train \
    --per_device_train_batch_size={batch_size} \
    --learning_rate="5e-5" --warmup_steps="2" --weight_decay 0.2
print("--\nDONE")
print(f"If it worked, trained data should be in {output_folder}")

Finetuning openai/clip-vit-base-patch32 for 60 epochs with batch size 20, and then saving output to results.
10/09/2023 10:44:54 - WARNING - __main__ - Process rank: 0, device: mps, n_gpu: 1distributed training: True, 16-bits training: False
Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 551.59it/s]
Generating train split: 500 examples [00:00, 109340.56 examples/s]
Filter: 100%|████████████████████████| 500/500 [00:00<00:00, 5899.01 examples/s]
Running tokenizer on train dataset: 100%|█| 494/494 [00:00<00:00, 19333.64 examp
Parameter 'transform'=<function main.<locals>.transform_images at 0x15714d240> of the transform datasets.arrow_dataset.Dataset.set_format couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompu

In [169]:
!zip -r archaeai.zip results/pytorch_model.bin results/config.json

updating: results/pytorch_model.bin (deflated 7%)
updating: results/config.json (deflated 46%)


In [170]:
from google.colab import files
files.download("archaeai.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

GPT4's 'insight' re the warning in the training script:

> That warning occurs because Python's `pickle` module is not able to serialize the function passed to `.map()`. In this case, the function is `transform_images`.

> Because the function `transform_images` is defined within the `main()` function, it technically includes references to all the local variables in `main()`, making it a closure. Closures are not serializable with `pickle`.

> A way around this would be to move the `transform_images` function out of the scope of the `main()` function. This means the function would be more of a "global" function rather than a local one, and it should remove the warning.

> Here's how you could modify it:

```python
def transform_images(examples, image_column, image_transformations):
    images = [read_image(image_file, mode=ImageReadMode.RGB) for image_file in examples[image_column]]
    examples["pixel_values"] = [image_transformations(image) for image in images]
    return examples
```

> And then within your main function, you can use it within the `.map()` function as:

```python
train_dataset.set_transform(lambda examples: transform_images(examples, image_column, image_transformations))
eval_dataset.set_transform(lambda examples: transform_images(examples, image_column, image_transformations))
```

>By passing the parameters explicitly and moving the function definition outside of the `main` function, it ensures that the function can be serialized and cached properly.